### Basic Setup
Here we check the sage version, configure some settings, define a 2-manifold $M$ with cylidrical coordinate chart $Y$, and a riemannian metric. The naming convention python variables (not variables used in the actual expressions) is `CamelCase` for objects defined on the manifold, and `snake_case` (with a trailing underscore for functions) for generic symbolic expressions. For rank $(0, 2)$ tensors, we default to the purely covariant form (as that is what we have regularization conditions for), denote the $(1, 1)$ version as `X_mat` and denote the rank $(2, 0)$ version as `X_con`.

In [11]:
# Check version
version()

'SageMath version 10.3, Release Date: 2024-03-19'

In [12]:
# Reset
reset()
# Setup pretty printing
%display latex

In [13]:
# Define a differentiable manifold of dimension 2 over real numbers
M = Manifold(2, 'M', latex_name=r'\mathcal{M}', start_index=1)
# Define a cylindrical chart on the manifold
Chart.<rho, z> = M.chart(r'rho:\rho z:z')
# Cache frame
Chartf = Chart.frame()

In [35]:
# Declare a metric on the manifold
G = M.riemannian_metric('g')

psi_ = function('psi_f', latex_name=r'\psi')(rho, z)
s_ = function('seed', latex_name='S')(rho, z)

sigma_ = function('sigma_f', latex_name=r'\sigma')(rho, z)

# Metric is symmetric
conformal_ = psi_**4 * exp(2 * rho * s_)

G[1, 1], G[2, 2] = conformal_, conformal_
G[1, 2] = 0

# Invert metric
G_con = G.inverse()

# Print metric
G.display()

g = e^(2*rho*seed(rho, z))*psi_f(rho, z)^4 drho⊗drho + e^(2*rho*seed(rho, z))*psi_f(rho, z)^4 dz⊗dz

In [36]:
G_con.display()

inv_g = e^(-2*rho*seed(rho, z))/psi_f(rho, z)^4 ∂/∂rho⊗∂/∂rho + e^(-2*rho*seed(rho, z))/psi_f(rho, z)^4 ∂/∂z⊗∂/∂z

In [37]:
# Compute Ricci Tensor
R = G.ricci()
# And Ricci Scalar
R_trace = G_con['^ij'] * R['_ij']
# And connection coefficients, used for covariant derivative
Nabla = G.connection(name='nabla', latex_name=r'\nabla')
# Make sure metric is compatible
assert(Nabla(G) == 0)

Nabla.display()

Gam^rho_rho,rho = (rho*psi_f(rho, z)*d(seed)/drho + psi_f(rho, z)*seed(rho, z) + 2*d(psi_f)/drho)/psi_f(rho, z) 
Gam^rho_rho,z = (rho*psi_f(rho, z)*d(seed)/dz + 2*d(psi_f)/dz)/psi_f(rho, z) 
Gam^rho_z,rho = (rho*psi_f(rho, z)*d(seed)/dz + 2*d(psi_f)/dz)/psi_f(rho, z) 
Gam^rho_z,z = -(rho*psi_f(rho, z)*d(seed)/drho + psi_f(rho, z)*seed(rho, z) + 2*d(psi_f)/drho)/psi_f(rho, z) 
Gam^z_rho,rho = -(rho*psi_f(rho, z)*d(seed)/dz + 2*d(psi_f)/dz)/psi_f(rho, z) 
Gam^z_rho,z = (rho*psi_f(rho, z)*d(seed)/drho + psi_f(rho, z)*seed(rho, z) + 2*d(psi_f)/drho)/psi_f(rho, z) 
Gam^z_z,rho = (rho*psi_f(rho, z)*d(seed)/drho + psi_f(rho, z)*seed(rho, z) + 2*d(psi_f)/drho)/psi_f(rho, z) 
Gam^z_z,z = (rho*psi_f(rho, z)*d(seed)/dz + 2*d(psi_f)/dz)/psi_f(rho, z)

In [45]:
R_trace.display()

M → ℝ
(rho, z) ↦ -2*(rho*psi_f(rho, z)^2*d^2(seed)/drho^2 + rho*psi_f(rho, z)^2*d^2(seed)/dz^2 + 2*psi_f(rho, z)^2*d(seed)/drho - 2*(d(psi_f)/drho)^2 + 2*psi_f(rho, z)*d^2(psi_f)/drho^2 - 2*(d(psi_f)/dz)^2 + 2*psi_f(rho, z)*d^2(psi_f)/dz^2)*e^(-2*rho*seed(rho, z))/psi_f(rho, z)^6

### Variables
Next we define the relavent tensor fields, as well as their components in our default frame.

In [38]:
# Resulting Tensors
Lam = M.scalar_field(rho * psi_^2, chart=Chart, name='lam', latex_name=r'\lambda')
          
Lam.display()

lam: M → ℝ
   (rho, z) ↦ rho*psi_f(rho, z)^2

In [39]:
Sigma = M.scalar_field(sigma_, chart=Chart, name='sigma_s', latex_name=r'\sigma')

Sigma.display()

sigma_s: M → ℝ
   (rho, z) ↦ sigma_f(rho, z)

In [40]:
# Gauge Tensors
Lapse = M.scalar_field(SR(1), chart=Chart, name='Lapse', latex_name=r'\alpha')

Shift = M.tensor_field(1, 0, name='Shift', latex_name=r'\beta')
Shift.add_comp(Chartf)[1] = SR(0)
Shift.add_comp(Chartf)[2] = SR(0)

Lapse.display()

Lapse: M → ℝ
   (rho, z) ↦ 1

In [41]:
Shift.display()

Shift = 0

In [42]:
# Extrinsic Curvature
K = M.tensor_field(0, 2, sym=(0, 1), name='K')
K.add_comp(Chartf)[1, 1] = SR(0)
K.add_comp(Chartf)[2, 2] = SR(0)
K.add_comp(Chartf)[1, 2] = SR(0)

K_mat = G_con['^{ij}'] * K['_jk']
K_con = K_mat['^i_k'] * G_con['^{kj}']
K_trace = G_con['^ij'] * K['_ij']

# L = K_\phi^\phi
L = M.scalar_field(SR(0), name='L')

K.display()

K = 0

In [43]:
L.display()

L: M → ℝ
   (rho, z) ↦ 0

### Constraint equations
Constraint equations that should be satisfied at all times.
$$ \mathcal{C} \equiv \frac{1}{2} (K^2 - K_{ij} K^{ij} + R) - \lambda^{-1} \nabla^j \nabla_j \lambda  + K L -\kappa \sigma =  0$$

$$ \mathcal{C}_i \equiv \nabla_j K_{i}^{\;j} - \nabla_i (K + L) + \lambda^{-1} (\nabla_j \lambda) K_{i}^{\;j} - \lambda^{-1} (\nabla_i \lambda) L - \kappa S_a= 0$$

In [44]:
term1 = (K_trace^2 - K['_ij'] * K_con['^ij'] + R_trace) / 2
term2 = - (Nabla(Nabla(Lam)) / Lam)['_ij'] * G_con['^{ij}'] + K_trace * L

# Hamiltonian
CH = term1 + term2 - Sigma

(CH.expr() * -conformal_ * psi_).expand()

e^(2*rho*seed(rho, z))*psi_f(rho, z)^5*sigma_f(rho, z) + rho*psi_f(rho, z)*diff(seed(rho, z), rho, rho) + rho*psi_f(rho, z)*diff(seed(rho, z), z, z) + 2*psi_f(rho, z)*diff(seed(rho, z), rho) + 4*diff(psi_f(rho, z), rho)/rho + 4*diff(psi_f(rho, z), rho, rho) + 4*diff(psi_f(rho, z), z, z)

In [25]:
# term1 = Nabla(K)['_ijk'] * G_con['^jk'] - Nabla(K_trace + L)
# term2 = LamLog['_j'] * K_mat['^j_i'] - LamLog * L

term1 = Nabla(K)['_ijk'] * G_con['^jk'] - Nabla(K_trace + L)
term2 = (Nabla(Lam) / Lam)['_i'] * K_mat['^i_j'] - (Nabla(Lam) / Lam) * L

# Momentum Constraint
CM = term1 + term2

CM.display()

0

### Preprocessing
To avoid transcription errors when transferring between notebooks and code we automatically transform symbolic expressions into `C` code.

In [47]:
# Symbolic Variables to replace current functions
var('psi_r psi_z psi_rr psi_zz psi_rz psi')
var('s_r s_z s_rr s_zz s_rz s')

"""
Preprocesses an expression, replacing all derivatives of a function
and invokations of that functions with an appropriately named variable.
"""
def process_expr(expr):
    expr = expr.simplify_full()
    derives = expr.subs({
        diff(psi_, rho): psi_r,
        diff(psi_, z): psi_z,
        diff(psi_, rho, rho): psi_rr,
        diff(psi_, z, z): psi_zz,
        diff(psi_, rho, z): psi_rz,
        diff(psi_, z, rho): psi_rz,
        
        diff(s_, rho): s_r,
        diff(s_, z): s_z,
        diff(s_, rho, rho): s_rr,
        diff(s_, z, z): s_zz,
        diff(s_, rho, z): s_rz,
        diff(s_, z, rho): s_rz,
    })
    
    values = derives.subs({
        psi_: psi,
        s_: s,
    })
    
    return values

process_expr(CH.expr() * psi_^5).expand()

-psi*rho*s_rr*e^(-2*rho*s) - psi*rho*s_zz*e^(-2*rho*s) - 2*psi*s_r*e^(-2*rho*s) - 4*psi_rr*e^(-2*rho*s) - 4*psi_zz*e^(-2*rho*s) - 4*psi_r*e^(-2*rho*s)/rho

In [48]:
def regularize(expr):
    sym_expr = expr._sympy_().expand()
    sym_expr = sym_expr.expand().subs([
        (s / rho, s_r),
        (psi_r / rho, psi_rr),
    ])
    # We should have removed all 1/r terms. If we didn't, there are now infs
    sym_expr = sym_expr.expand().subs(rho, 0)
    sym_expr = sym_expr.expand().subs([
        (s, 0),
        (psi_r, 0),
    ])
    sym_expr = sym_expr.expand().subs([
        (s_z, 0),
        (psi_rz, 0),
    ])
    
    return sym_expr.expand()._sage_()

# Term = (Nabla(Lam) / Lam)['_i'] * K_mat['^i_j'] - Nabla(Lam) / Lam * L

# expr = process_expr(Term[1].expr()).expand()
# expr

In [49]:
import sympy as sym

def generate_ccode(eqs):
    names = []
    exprs = []
    
    for (name, eq) in eqs:
        names.append(name)
        exprs.append(process_expr(eq)._sympy_().simplify())
        
    subs, final = sym.cse(exprs)
    
    result = ("/*************************************\n" + 
              "This code was generated automatically\n" +
              "using Sagemath and SymPy\n" +
              "**************************************/\n")
    result += "\n// Subexpressions\n"
    
    for (name, expr) in subs:
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n"
        
    result += "\n// Final Equations\n"
        
    for (name, expr) in zip(names, final):
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n\n"
        
    return result

def generate_regular_ccode(eqs):
    names = []
    exprs = []
    
    for (name, eq) in eqs:
        names.append(name)
        exprs.append(regularize(process_expr(eq))._sympy_().simplify())
        
    subs, final = sym.cse(exprs)
    
    result = ("/*************************************\n" + 
              "This code was generated automatically\n" +
              "using Sagemath and SymPy\n" +
              "**************************************/\n")
    result += "\n// Subexpressions\n"
    
    for (name, expr) in subs:
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n"
        
    result += "\n// Final Equations\n"
        
    for (name, expr) in zip(names, final):
        code = sym.ccode(expr)
        result += "double " + str(name) + " = " + str(code) + ";\n\n"
        
    return result

In [52]:
initial_op = (-CH.expr() * psi_^5  * exp(2 * s_ * rho)).expand()

code = generate_ccode([
    ("op", initial_op),
])

initial_h = open("initial.h", 'w')
initial_h.write(code)
initial_h.close()

print(code)

/*************************************
This code was generated automatically
using Sagemath and SymPy
**************************************/

// Subexpressions
double x0 = psi*rho;

// Final Equations
double op = 2*psi*s_r + 4*psi_r/rho + 4*psi_rr + 4*psi_zz + s_rr*x0 + s_zz*x0;




In [51]:
code = generate_regular_ccode([
    ("op", initial_op),
])

initial_h = open("initial_regular.h", 'w')
initial_h.write(code)
initial_h.close()

print(code)

/*************************************
This code was generated automatically
using Sagemath and SymPy
**************************************/

// Subexpressions

// Final Equations
double op = 2*psi*s_r + 8*psi_rr + 4*psi_zz;


